Introduction

In [1]:
#Imports
import requests
import pandas as pd
import numpy as np
import os

In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d4b60abdfea34838956c197ac90f60c3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9LXqtVfcHVPyIcCt1uVbAOCEdKljW6v4cLcMMhqwNnMH',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d4b60abdfea34838956c197ac90f60c3.get_object(Bucket='capstoneweek5-donotdelete-pr-povvigwqhpfbjy',Key='zomato.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_food = pd.read_csv(body,encoding = "ISO-8859-1")
df_food.head()



,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [3]:
list(df_food)

['Restaurant ID',
 'Restaurant Name',
 'Country Code',
 'City',
 'Address',
 'Locality',
 'Locality Verbose',
 'Longitude',
 'Latitude',
 'Cuisines',
 'Average Cost for two',
 'Currency',
 'Has Table booking',
 'Has Online delivery',
 'Is delivering now',
 'Switch to order menu',
 'Price range',
 'Aggregate rating',
 'Rating color',
 'Rating text',
 'Votes']

In [4]:
df_food_clean = df_food[['City','Longitude','Latitude','Cuisines']]
df_food_clean.head()

,City,Longitude,Latitude,Cuisines
0,Makati City,121.027535,14.565443,"French, Japanese, Desserts"
1,Makati City,121.014101,14.553708,Japanese
2,Mandaluyong City,121.056831,14.581404,"Seafood, Asian, Filipino, Indian"
3,Mandaluyong City,121.056475,14.585318,"Japanese, Sushi"
4,Mandaluyong City,121.057508,14.584450,"Japanese, Korean"


In [5]:
indexNames = df_food_clean[ df_food_clean['City'] != 'New Delhi'].index
df_food_clean.drop(indexNames , inplace=True)
df_food_clean.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,City,Longitude,Latitude,Cuisines
2560,New Delhi,0.000000,0.000000,Cuisine Varies
2561,New Delhi,77.196923,28.535382,Fast Food
2562,New Delhi,77.197475,28.535493,"North Indian, Seafood, Continental"
2563,New Delhi,77.198033,28.537547,"South Indian, North Indian"
2564,New Delhi,77.196924,28.535523,"South Indian, North Indian, Chinese"


Visualising non Mexican shops in Gurgaon

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [7]:
#Gurgaon

In [8]:
df_food_clean = df_food[['City','Longitude','Latitude','Cuisines']]
df_food_clean.head()
df_food_clean_gurgaon = df_food_clean
df_food_clean_gurgaon.head()

,City,Longitude,Latitude,Cuisines
0,Makati City,121.027535,14.565443,"French, Japanese, Desserts"
1,Makati City,121.014101,14.553708,Japanese
2,Mandaluyong City,121.056831,14.581404,"Seafood, Asian, Filipino, Indian"
3,Mandaluyong City,121.056475,14.585318,"Japanese, Sushi"
4,Mandaluyong City,121.057508,14.584450,"Japanese, Korean"


In [9]:
indexNames = df_food_clean_gurgaon[ df_food_clean_gurgaon['City'] != 'Gurgaon'].index
df_food_clean_gurgaon.drop(indexNames , inplace=True)
df_food_clean_gurgaon.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,City,Longitude,Latitude,Cuisines
1161,Gurgaon,77.039310,28.424831,"Cafe, Beverages"
1162,Gurgaon,77.039220,28.424913,"North Indian, Mughlai"
1163,Gurgaon,77.097118,28.503077,"North Indian, Mughlai"
1164,Gurgaon,77.097185,28.502613,"South Indian, Seafood, Kerala"
1165,Gurgaon,77.042009,28.511416,Bakery


In [10]:
to_remove = ['Mexican','Cafe','Beverages','Bakery','Ice','Fast','Indian','Raw','Tex','Healthy','Street','Drinks','Desserts']  
df_non_mexican_gurgaon= df_food_clean_gurgaon
for word in to_remove:
    df_non_mexican_gurgaon =df_non_mexican_gurgaon[~df_non_mexican_gurgaon['Cuisines'].str.contains(word)]
df_non_mexican_gurgaon.head()

,City,Longitude,Latitude,Cuisines
1167,Gurgaon,77.042009,28.511506,Chinese
1169,Gurgaon,77.087851,28.440709,"Japanese, Thai, Chinese"
1188,Gurgaon,77.034715,28.458088,Japanese
1191,Gurgaon,77.100662,28.443407,Biryani
1194,Gurgaon,77.100737,28.443110,"Italian, Pizza, Lebanese"


In [11]:
df_non_mexican_gurgaon.shape

(194, 4)

In [12]:
df_non_mexican_gurgaon['comma_count'] = df_non_mexican_gurgaon.Cuisines.str.count(',') #0 means 1
df_non_mexican_gurgaon.head()

,City,Longitude,Latitude,Cuisines,comma_count
1167,Gurgaon,77.042009,28.511506,Chinese,0
1169,Gurgaon,77.087851,28.440709,"Japanese, Thai, Chinese",2
1188,Gurgaon,77.034715,28.458088,Japanese,0
1191,Gurgaon,77.100662,28.443407,Biryani,0
1194,Gurgaon,77.100737,28.443110,"Italian, Pizza, Lebanese",2


In [13]:
indexNames = df_non_mexican_gurgaon[ df_non_mexican_gurgaon['comma_count'] > 0].index
df_non_mexican_gurgaon.drop(indexNames , inplace=True)
df_non_mexican_gurgaon.head()

,City,Longitude,Latitude,Cuisines,comma_count
1167,Gurgaon,77.042009,28.511506,Chinese,0
1188,Gurgaon,77.034715,28.458088,Japanese,0
1191,Gurgaon,77.100662,28.443407,Biryani,0
1195,Gurgaon,77.101096,28.442696,Lebanese,0
1202,Gurgaon,77.037691,28.468336,Finger Food,0


In [14]:
df_non_mexican_gurgaon.shape

(106, 5)

In [15]:
#foursquare
address = 'Gurgaon, In'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Gurgaon are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Gurgaon are 28.4646148, 77.0299194.


In [16]:
df_non_mexican_gurgaon.Cuisines.unique()

array(['Chinese', 'Japanese', 'Biryani', 'Lebanese', 'Finger Food',
       'Kashmiri', 'South American', 'Italian', 'Mughlai', 'Bengali',
       'Lucknowi', 'Thai', 'Pizza', 'Korean', 'Mithai', 'Oriya', 'Tea',
       'Afghani', 'Continental', 'Mediterranean', 'Asian', 'Juices',
       'Goan', 'Rajasthani'], dtype=object)

In [17]:
# create map of Gurgaon using latitude and longitude values
map_gurgaon = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, City, Cuisines in zip(df_non_mexican_gurgaon['Latitude'], df_non_mexican_gurgaon['Longitude'], df_non_mexican_gurgaon['City'], df_non_mexican_gurgaon['Cuisines']):
    label = '{}'.format( Cuisines)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gurgaon)  
    


In [19]:
map_gurgaon

In [21]:
#MAIN
CLIENT_ID = 'VBRAB5AE2HIRMN5PYEIE3FLGX2EPEUHWDP0PMXKLRUFY4SYV' # your Foursquare ID
CLIENT_SECRET = '4MK2KHERDRQ0KTLD3DRLF5LURPDBB2UIIB0MLRXZ4SKBXVYG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VBRAB5AE2HIRMN5PYEIE3FLGX2EPEUHWDP0PMXKLRUFY4SYV
CLIENT_SECRET:4MK2KHERDRQ0KTLD3DRLF5LURPDBB2UIIB0MLRXZ4SKBXVYG


In [48]:
CLIENT_ID = 'RBN5OE2NQPXVAQPUE4CYQXU1BWJT21B2CRDVFCWFWIFETLU0' # your Foursquare ID
CLIENT_SECRET = 'AGIWQEEFOBPLMIVEE1QGLQP0E0ZEE3ZTUFVPLDQBNMCA1CWP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RBN5OE2NQPXVAQPUE4CYQXU1BWJT21B2CRDVFCWFWIFETLU0
CLIENT_SECRET:AGIWQEEFOBPLMIVEE1QGLQP0E0ZEE3ZTUFVPLDQBNMCA1CWP


In [49]:
df_non_mexican_gurgaon.head()

,City,Longitude,Latitude,Cuisines,comma_count
1167,Gurgaon,77.042009,28.511506,Chinese,0
1188,Gurgaon,77.034715,28.458088,Japanese,0
1191,Gurgaon,77.100662,28.443407,Biryani,0
1195,Gurgaon,77.101096,28.442696,Lebanese,0
1202,Gurgaon,77.037691,28.468336,Finger Food,0


In [50]:
neighborhood_latitude = df_non_mexican_gurgaon.loc[1167, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_non_mexican_gurgaon.loc[1167, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_non_mexican_gurgaon.loc[1167, 'Cuisines'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(cuisines_name, 
                                                               cuisines_latitude, 
                                                               cuisines_longitude))

Latitude and longitude values of Chinese are 28.5115059, 77.04200909999999.


In [51]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    cuisines_latitude, 
    cuisines_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RBN5OE2NQPXVAQPUE4CYQXU1BWJT21B2CRDVFCWFWIFETLU0&client_secret=AGIWQEEFOBPLMIVEE1QGLQP0E0ZEE3ZTUFVPLDQBNMCA1CWP&v=20180605&ll=28.5115059,77.04200909999999&radius=500&limit=100'

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7d270d4c1f67636df8fd0f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ca0ba118afca09385351516-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '4ca0ba118afca09385351516',
       'location': {'address': '13, Ground Floor, Ansal Plaza Mall',
        'cc': 'IN',
        'city': 'Gurgaon',
        'country': 'India',
        'crossStreet': 'Palam Vihar',
        'distance': 43,
        'formattedAddress': ['13, Ground Floor, Ansal Plaza Mall (Palam Vihar)',
         'Gurgaon',
         'Haryāna',
         'India'],
        

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Cuisines', 
                  'Cuisines Latitude', 
                  'Cuisines Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
df_non_mexican_gurgaon_new =df_non_mexican_gurgaon.iloc[80:]
df_non_mexican_gurgaon_new.shape

(26, 5)

In [57]:
Gurgaon_venues = getNearbyVenues(names=df_non_mexican_gurgaon_new['Cuisines'],
                                   latitudes=df_non_mexican_gurgaon_new['Latitude'],
                                   longitudes=df_non_mexican_gurgaon_new['Longitude']
                                  )

Tea
Italian
Asian
Biryani
Continental
Juices
Biryani
Lebanese
Goan
Biryani
Rajasthani
Chinese
Continental
Biryani
Asian
Finger Food
Japanese
Finger Food
Italian
Chinese
Continental
Finger Food
Japanese
Chinese
Chinese
Mithai


In [58]:
print(Gurgaon_venues.shape)
Gurgaon_venues.head()

(273, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tea,28.482879,77.095474,Plan B,28.484890,77.096439,Speakeasy
1,Tea,28.482879,77.095474,"Sikanderpur Market, Gurgaon",28.481883,77.095322,Furniture / Home Store
2,Tea,28.482879,77.095474,DT CINEMAS,28.483537,77.094018,Multiplex
3,Tea,28.482879,77.095474,Bristol Hotel,28.479430,77.095101,Hotel
4,Italian,28.406072,77.044708,Open Tap,28.405130,77.044332,Brewery


In [59]:
Gurgaon_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Asian,28,28,28,28,28,28
Biryani,52,52,52,52,52,52
Chinese,28,28,28,28,28,28
Continental,39,39,39,39,39,39
Finger Food,41,41,41,41,41,41
Goan,12,12,12,12,12,12
Italian,22,22,22,22,22,22
Japanese,19,19,19,19,19,19
Juices,5,5,5,5,5,5


In [60]:
print('There are {} uniques categories.'.format(len(Gurgaon_venues['Venue Category'].unique())))

There are 56 uniques categories.


In [61]:
# one hot encoding
Gurgaon_onehot = pd.get_dummies(Gurgaon_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Gurgaon_onehot['Cuisines'] = Gurgaon_venues['Cuisines'] 

# move neighborhood column to the first column
fixed_columns = [Gurgaon_onehot.columns[-1]] + list(Gurgaon_onehot.columns[:-1])
Gurgaon_onehot = Gurgaon_onehot[fixed_columns]

Gurgaon_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Breakfast Spot,Brewery,Burger Joint,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Fast Food Restaurant,Flower Shop,Food,Food Truck,Furniture / Home Store,Grocery Store,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Liquor Store,Lounge,Market,Middle Eastern Restaurant,Movie Theater,Mughlai Restaurant,Multiplex,Nightlife Spot,Pizza Place,Plaza,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Speakeasy,Supermarket,Tea Room,Tennis Court,Thai Restaurant,Wine Bar
0,Tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Italian,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [62]:
Gurgaon_onehot.shape

(273, 57)

In [63]:
Gurgaon_grouped = Gurgaon_onehot.groupby('Cuisines').mean().reset_index()
Gurgaon_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Breakfast Spot,Brewery,Burger Joint,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Fast Food Restaurant,Flower Shop,Food,Food Truck,Furniture / Home Store,Grocery Store,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Liquor Store,Lounge,Market,Middle Eastern Restaurant,Movie Theater,Mughlai Restaurant,Multiplex,Nightlife Spot,Pizza Place,Plaza,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Speakeasy,Supermarket,Tea Room,Tennis Court,Thai Restaurant,Wine Bar
0,Asian,0.035714,0.00000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.00000,0.000000,0.035714,0.000000,0.142857,0.071429,0.000000,0.178571,0.000000,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.00000,0.000000,0.000000,0.000000,0.035714,0.071429,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Biryani,0.000000,0.00000,0.000000,0.000000,0.038462,0.000000,0.019231,0.000000,0.000000,0.019231,0.000000,0.096154,0.038462,0.019231,0.000000,0.000000,0.019231,0.000000,0.019231,0.000000,0.038462,0.000000,0.019231,0.000000,0.019231,0.000000,0.00000,0.000000,0.038462,0.019231,0.173077,0.076923,0.038462,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.019231,0.057692,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Chinese,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107143,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.035714,0.035714,0.071429,0.035714,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.035714,0.035714,0.035714,0.107143,0.000000,0.000000,0.035714,0.035714,0.000000,0.000000,0.000000,0.00000,0.000000,0.071429,0.035714,0.071429,0.035714,0.000000,0.035714,0.000000,0.00,0.000000,0.035714,0.035714,0.035714,0.000000
3,Continental,0.000000,0.00000,0.000000,0.000000,0.051282,0.000000,0.025641,0.000000,0.000000,0.025641,0.000000,0.128205,0.051282,0.025641,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.025641,0.000000,0.051282,0.000000,0.025641,0.000000,0.00000,0.000000,0.025641,0.000000,0.128205,0.051282,0.051282,0.128205,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.025641,0.051282,0.000000,0.025641,0.025641,0.00,0.025641,0.000000,0.000000,0.000000,0.000000
4,Finger Food,0.024390,0.02439,0.000000,0.000000,0.000000,0.048780,0.000000,0.000000,0.024390,0.000000,0.048780,0.024390,0.024390,0.000000,0.024390,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.048780,0.000000,0.000000,0.000000,0.02439,0.000000,0.024390,0.000000,0.170732,0.000000,0.000000,0.073171,0.000000,0.000000,0.048780,0.000000,0.000000,0.000000,0.000000,0.02439,0.048780,0.024390,0.000000,0.121951,0.048780,0.048780,0.024390,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Goan,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.083333,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.250000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Italian,0.000000,0.00000,0.000000,0.045455,0.045455,0.000000,0.000000,0.045455,0.090909,0.000000,0.000000,0.045455,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [64]:
Gurgaon_grouped.shape

(13, 57)

In [65]:
num_top_venues = 5

for hood in Gurgaon_grouped['Cuisines']:
    print("----"+hood+"----")
    temp = Gurgaon_grouped[Gurgaon_grouped['Cuisines'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Asian----
               venue  freq
0  Indian Restaurant  0.18
1              Hotel  0.14
2     Sandwich Place  0.07
3             Bakery  0.07
4            Brewery  0.07


----Biryani----
               venue  freq
0              Hotel  0.17
1  Indian Restaurant  0.15
2               Café  0.10
3          Hotel Bar  0.08
4        Snack Place  0.06


----Chinese----
                  venue  freq
0     Indian Restaurant  0.11
1                  Café  0.11
2            Restaurant  0.07
3  Fast Food Restaurant  0.07
4           Pizza Place  0.07


----Continental----
               venue  freq
0  Indian Restaurant  0.13
1              Hotel  0.13
2               Café  0.13
3     Sandwich Place  0.05
4     Ice Cream Shop  0.05


----Finger Food----
                  venue  freq
0                 Hotel  0.17
1            Restaurant  0.12
2     Indian Restaurant  0.07
3  Fast Food Restaurant  0.05
4    Seafood Restaurant  0.05


----Goan----
                  venue  freq
0     Indian Re

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
   
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cuisines']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Cuisines'] = Gurgaon_grouped['Cuisines']

for ind in np.arange(Gurgaon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Gurgaon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Asian,Indian Restaurant,Hotel,Café,Bakery,Hotel Bar,Sandwich Place,Brewery,American Restaurant,Diner,Gym
1,Biryani,Hotel,Indian Restaurant,Café,Hotel Bar,Snack Place,Sandwich Place,Ice Cream Shop,Restaurant,Chinese Restaurant,Gym
2,Chinese,Indian Restaurant,Café,Fast Food Restaurant,Restaurant,Pizza Place,Market,Lounge,Diner,Donut Shop,Flower Shop
3,Continental,Hotel,Indian Restaurant,Café,Fast Food Restaurant,Hotel Bar,Ice Cream Shop,Bakery,Sandwich Place,Chinese Restaurant,Burger Joint
4,Finger Food,Hotel,Restaurant,Indian Restaurant,Fast Food Restaurant,Cafeteria,Bar,Seafood Restaurant,Sandwich Place,Nightlife Spot,Lounge


In [70]:
# set number of clusters
kclusters = 5

mexican_grouped_clustering = Gurgaon_grouped.drop('Cuisines', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mexican_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 4, 0], dtype=int32)

In [78]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mexican_merged = df_non_mexican_gurgaon_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mexican_merged = mexican_merged.join(neighborhoods_venues_sorted.set_index('Cuisines'), on='Cuisines')

mexican_merged.head() # check the last columns!

,City,Longitude,Latitude,Cuisines,comma_count,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2044,Gurgaon,77.095474,28.482879,Tea,0,1,Furniture / Home Store,Speakeasy,Hotel,Multiplex,Clothing Store,Food,Flower Shop,Fast Food Restaurant,Donut Shop,Diner
2070,Gurgaon,77.044708,28.406072,Italian,0,0,Indian Restaurant,Italian Restaurant,Brewery,Comfort Food Restaurant,Coffee Shop,Hotel Bar,Hotel,Café,Mughlai Restaurant,Gym
2076,Gurgaon,77.041379,28.406918,Asian,0,0,Indian Restaurant,Hotel,Café,Bakery,Hotel Bar,Sandwich Place,Brewery,American Restaurant,Diner,Gym
2080,Gurgaon,77.042099,28.412009,Biryani,0,0,Hotel,Indian Restaurant,Café,Hotel Bar,Snack Place,Sandwich Place,Ice Cream Shop,Restaurant,Chinese Restaurant,Gym
2087,Gurgaon,77.041790,28.415939,Continental,0,0,Hotel,Indian Restaurant,Café,Fast Food Restaurant,Hotel Bar,Ice Cream Shop,Bakery,Sandwich Place,Chinese Restaurant,Burger Joint


In [81]:
mexican_merged['Cuisines'].unique()

array(['Tea', 'Italian', 'Asian', 'Biryani', 'Continental', 'Juices',
       'Lebanese', 'Goan', 'Rajasthani', 'Chinese', 'Finger Food',
       'Japanese', 'Mithai'], dtype=object)

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mexican_merged['Latitude'], mexican_merged['Longitude'], mexican_merged['Cuisines'], mexican_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        color = 'red',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [85]:
#MEXICAN FOOD ONLY
#to_remove = ['Mexican','Cafe','Beverages','Bakery','Ice','Fast','Indian','Raw','Tex','Healthy','Street','Drinks','Desserts']  
df_mexican_gurgaon= df_food_clean_gurgaon
for word in to_remove:
    df_mexican_gurgaon =df_mexican_gurgaon[df_mexican_gurgaon['Cuisines'].str.contains("Mexican")]
df_mexican_gurgaon.head()

,City,Longitude,Latitude,Cuisines
1219,Gurgaon,77.088242,28.494757,"American, Mexican"
1243,Gurgaon,77.088311,28.494264,"Mexican, Fast Food"
1253,Gurgaon,77.088508,28.494922,"Mediterranean, Mexican, North Indian, American"
1301,Gurgaon,77.092285,28.473322,"Chinese, Mexican, Fast Food, Continental"
1381,Gurgaon,77.103974,28.487264,"Mexican, American, Healthy Food"


In [86]:
# add markers to map
for lat, lng, City, Cuisines in zip(df_mexican_gurgaon['Latitude'], df_mexican_gurgaon['Longitude'], df_mexican_gurgaon['City'], df_mexican_gurgaon['Cuisines']):
    label = '{}'.format( Cuisines)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)

In [87]:
map_clusters